# Main

In [1]:
# Import Libraries

In [16]:
### Import Libraries

# File/OS/Internet Interface

import os # For working with Operating System
import sys # System arguments
import requests # Accessing the Web
from dotenv import load_dotenv # Loading .env info
import time # For Sleeping

# Database 

import psycopg2
from psycopg2 import sql

# Analysis

import datetime as dt # Working with dates/times
import pytz # Timezones
import numpy as np
import pandas as pd
import geopandas as gpd


In [59]:
## Load Functions

script_path = os.path.join('..', '..', 'Scripts', 'python')

# Function definition - Please see Scripts/python/*
exec(open(os.path.join(script_path, 'Get_spikes_df.py')).read())
exec(open(os.path.join(script_path, 'Create_messages.py')).read())
exec(open(os.path.join(script_path, 'twilio_functions.py')).read())
exec(open(os.path.join(script_path, 'Update_Alerts.py')).read())

In [17]:
### Definitions

load_dotenv() # Load .env file

## API Keys

purpleAir_api = os.getenv('PURPLEAIR_API_TOKEN') # PurpleAir API Read Key

redCap_token = os.getenv('REDCAP_TOKEN') # Surveys

## Database credentials

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))  

## Twilio Information

TWILIO_ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')

# Other Constants

spike_threshold = 35 # Value which defines an AQ_Spike (Micgrograms per meter cubed)

# When to stop the program?
days_to_run = 7 # How many days will we run this?
timestep = 10 # Sleep time in between updates (in Minutes)
stoptime = dt.datetime.now() + dt.timedelta(days=days_to_run) # When to stop the program (datetime)


In [63]:
### The Script

while True:

    now = dt.datetime.now() # The current time

    print(now)

    if stoptime < now: # Check if we've hit stoptime
        print("Terminating Program")
        break

    # Get the sensor_ids from sensors in our database

    sensor_ids = get_sensor_ids(pg_connection_dict) # In Get_Spikes_df.py

    # Check our active alerts

    active_alerts_df = get_active_alerts(pg_connection_dict) # In Update_Alerts.py

    # Query PurpleAir for Spikes

    spikes_df, runtime = Get_spikes_df(purpleAir_api, sensor_ids, spike_threshold) # In Get_Spikes_df.py

    # Sort the spiked sensors into new, ongoing, and not spiked sensors

    new_spike_sensors, ongoing_spike_sensors, ended_spike_sensors, not_spiked_sensors = sort_sensors_for_updates(spikes_df, sensor_ids, pg_connection_dict) # In Update_Alerts.py

    # NEW Spikes
    
    if len(new_spike_sensors) > 0:

        new_spikes_df = spikes_df[spikes_df.sensor_index.isin(new_spike_sensors)]
    
        for index, row in new_spikes_df.iterrows():
    
            # 1) Add to active alerts
        
            add_to_active_alerts(row, pg_connection_dict,
                                 runtime.strftime('%Y-%m-%d %H:%M:%S') # When we ran the PurpleAir Query
                                )

            # 2) Text users about this - NOT DONE
    
            # 3) Add to User's Active Alerts - NOT DONE

    # ONGOING spikes

    if len(ongoing_spike_sensors) > 0:

        ongoing_spikes_df = spikes_df[spikes_df.sensor_index.isin(ongoing_spike_sensors)]

        for _, spike in ongoing_spikes_df.iterrows():

            # 1) Update the maximum reading
    
            update_max_reading(spike, pg_connection_dict)

    # ENDED spikes

    if len(ended_spike_sensors) > 0:

        # 1) Add alert to archive - NEEDS WORK
    
        add_to_archived_alerts(not_spiked_sensors, pg_connection_dict)

        # 2) Remove from Active Alerts - NEEDS WORK
        
        ended_alert_indices = remove_active_alerts(not_spiked_sensors, pg_connection_dict)

        # 3) Text people it's over - NOT DONE

    # break

    # SLEEP

    time.sleep(timestep*60) # Sleep between updates

2023-10-26 01:13:47.619548


KeyboardInterrupt: 

In [1]:
active_alerts_df

NameError: name 'active_alerts_df' is not defined

In [65]:
spikes_df

,sensor_index,pm25
35,143944,79.0


In [66]:
new_spike_sensors, ongoing_spike_sensors, ended_spike_sensors, not_spiked_sensors

(set(), {143944}, set())